# AI for BI

Analyse data and provide a BI report including charts based on that data

We'll use CrewAI for the processing. This includes a Python interpreter which will be used to create the charts. You must have Docker installed locally to run automatically the code.


Import the necessary libraries and set the LLM model (defaults to using OpenAI)

In [1]:
from crewai import Agent, Task, Crew

llm = "gpt-4o-mini"

Import the tools required to read and write files

In [2]:
from crewai_tools import tool
from crewai_tools import FileReadTool, FileWriterTool

# Initialize the tool to read any files the agents knows or lean the path for
file_read_tool = FileReadTool()
file_writer_tool = FileWriterTool()

Set up the ``data_analysis_agent``

In [3]:
# Define the agent
data_analysis_agent = Agent(
        role="Data Analyser",
        goal="""You aim is to read and analyse sales data""",
        backstory="You are assigned to perform sales analysis for a company",
        tools=[file_read_tool, file_writer_tool],
        allow_delegation=False,
        llm=llm,
        allow_code_execution=True
    )

The following defines a function that will create a task to create a markdown summary and chart for each data file and set up a crew.

We define the files and then loop through them calling the ``create_charts`` function to write the individual markdown files.


In [4]:
def create_charts(query):
    create_chart = Task(
        description=query,
        expected_output="""A markdown summary and a matplotlib chart""",
        agent=data_analysis_agent,
        tools=[file_read_tool, file_writer_tool]
    )
    
    # Define the crew
    crew = Crew(
        agents=[data_analysis_agent],
        tasks=[create_chart],
        verbose=True
    )
    return crew, create_chart


files = [
    {
        'data_file_name':'./sales_product_cat.csv',
        'chart_file_name': './reports/sales_product_summary.png',
        'report_file_name': './reports/sales_product_summary.md',
        'local_report_name': 'sales_product_summary.md',
    },
    {
        'data_file_name': './monthly_sales.csv',
        'chart_file_name': './reports/monthly_sales.png',
        'report_file_name': './reports/monthly_sales.md',
        'local_report_name': 'monthly_sales.md',
    },
    {
        'data_file_name': './sales_by_region.csv',
        'chart_file_name': './reports/sales_by_region.png',
        'report_file_name': './reports/sales_by_region.md',
        'local_report_name': 'sales_by_region.md',
    }
]


for f in files:
    query = f"""Read the Sales by product category data from the file '{f['data_file_name']}',
                draw a chart that summarizes the data and save it to the file '{f['chart_file_name']}'
                and write a summary of the data in markdown that includes a reference to the chart in '{f['local_report_name']}'
                and save it in the file '{f['report_file_name']}'"""
    print(query)
    crew, task = create_charts(query)
    crew.kickoff()
    task_output = task.output


Read the Sales by product category data from the file './sales_product_cat.csv',
                draw a chart that summarizes the data and save it to the file './reports/sales_product_summary.png'
                and write a summary of the data in markdown that includes a reference to the chart in 'sales_product_summary.md'
                and save it in the file './reports/sales_product_summary.md'
# Agent: Data Analyser
## Task: Read the Sales by product category data from the file './sales_product_cat.csv',
                draw a chart that summarizes the data and save it to the file './reports/sales_product_summary.png'
                and write a summary of the data in markdown that includes a reference to the chart in 'sales_product_summary.md'
                and save it in the file './reports/sales_product_summary.md'


# Agent: Data Analyser
## Thought: I need to start by reading the sales data from the specified CSV file so that I can analyze it and create the chart.
## Using

2024-11-06 19:57:36,799 - 9584 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Data Analyser
## Final Answer: 
```markdown
# Sales Summary by Product Category

| Product Category | Units Sold | Revenue ($) |
|------------------|------------|--------------|
| Electronics      | 4700       | 2490000      |
| Furniture        | 700        | 290000       |
| Home Appliances   | 2100       | 735000       |

![Sales Summary Chart](./reports/sales_product_summary.png)
```

Additionally, the chart summarizing the total revenue by product category has been saved as './reports/sales_product_summary.png'.


Read the Sales by product category data from the file './monthly_sales.csv',
                draw a chart that summarizes the data and save it to the file './reports/monthly_sales.png'
                and write a summary of the data in markdown that includes a reference to the chart in 'monthly_sales.md'
                and save it in the file './reports/monthly_sales.md'
# Agent: Data Analyser
## Task: Read the Sales by product category data from the file './

2024-11-06 19:58:45,177 - 9584 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Data Analyser
## Final Answer: 
### Markdown Summary (monthly_sales.md)
```markdown
# Monthly Sales Summary

This report provides a summary of the total units sold and total revenue for the sales in the first eight months of the year.

| Month     | Total Units Sold | Total Revenue ($) | % Change (Month-over-Month) |
|-----------|-------------------|---------------------|-------------------------------|
| January   | 10000             | 3500000             | -                             |
| February  | 10500             | 3675000             | +5%                           |
| March     | 11000             | 3850000             | +4.8%                         |
| April     | 10800             | 3780000             | -1.8%                         |
| May       | 11500             | 4025000             | +6.5%                         |
| June      | 11300             | 3955000             | -1.7%                         |
| July      | 12000             | 4200000             

In [5]:


#print(f"Task Description: {task_output.description}")
#print(f"Task Summary: {task_output.summary}")
#print(f"Raw Output: {task_output.raw}")